# Информационный поиск

Заменяем текст запроса по заданию

In [1]:
QUERIES = ['compressibility transformation', 'yawed cylinder']

Скачиваем данные

In [2]:
! wget -q http://ir.dcs.gla.ac.uk/resources/test_collections/cran/cran.tar.gz
! tar -xvf cran.tar.gz
! rm cran.tar.gz*
! grep -v "^\." cran.qry > just.qry
! head -3 just.qry
! pip install -q scikit-learn==0.22.2.post1

cran.all.1400
cran.qry
cranqrel
cranqrel.readme
what similarity laws must be obeyed when constructing aeroelastic models
of heated high speed aircraft .
what are the structural and aeroelastic problems associated with flight
     |████████████████████████████████| 7.1 MB 5.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.22.2.post1 which is incompatible.


Запускаем код

In [3]:
from  sklearn.feature_extraction.text import CountVectorizer
from  sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import scipy.spatial.distance as ds 
import warnings

warnings.filterwarnings("ignore")

def cosine_distance(vector_a: np.array, vector_b: np.array) -> float:
  return ds.cosine(vector_a, vector_b)

def jaccard_sim(vector_a: np.array, vector_b: np.array) -> float:
  vector_a = np.asarray(vector_a, np.bool)
  vector_b = np.asarray(vector_b, np.bool)
  return np.double(np.bitwise_and(vector_a, vector_b).sum()) / np.double(np.bitwise_or(vector_a, vector_b).sum())

raw_query_data = [line.strip() for line in open("just.qry", "r").readlines()]
query_data = [""]

for query_part in raw_query_data:
  query_data[-1] += query_part + " "
  if query_part.endswith("."):
    query_data.append("")

encoder = CountVectorizer(binary=True)
encoded_data = encoder.fit_transform(query_data)
encoded_queries = encoder.transform(QUERIES)

id2term = {idx: term for term, idx in encoder.vocabulary_.items()}
non_zero_values_ids = encoded_data[0].nonzero()[1]

terms = [id2term[idx] for idx in non_zero_values_ids]

print("По мера Жаккара:")
for q_id, query in enumerate(encoded_queries):
  query = query.todense().A1
  docs = [doc.todense().A1 for doc in encoded_data]
  id2doc2similarity = [(doc_id, doc, jaccard_sim(query, doc)) for doc_id, doc in enumerate(docs)]
  closest = sorted(id2doc2similarity, key=lambda x: x[2], reverse=True)
  
  print("Q: %s:" %(QUERIES[q_id]))
  print("    %s\t%s" %("ID", "Коэффициент"))
  for closest_id, _, sim in closest[:2]:
    print("    %d\t%.2f" %(closest_id, sim))

print()

# Второе задание
tfidf_encoder = TfidfVectorizer()
tfidf_encoded_data = tfidf_encoder.fit_transform(query_data)
tfidf_encoded_queries = tfidf_encoder.transform(QUERIES)

print("Косинустное расстояние:")
for q_id, query in enumerate(tfidf_encoded_queries):
  query = query.todense().A1
  docs = [doc.todense().A1 for doc in tfidf_encoded_data]
  id2doc2similarity = [(doc_id, doc, cosine_distance(query, doc)) for doc_id, doc in enumerate(docs)]
  closest = sorted(id2doc2similarity, key=lambda x: x[2], reverse=False)

  print("Q: %s:" %(QUERIES[q_id]))
  print("    %s\t%s" %("ID", "Коэффициент"))
  for closest_id, _, sim in closest[:2]:
    print("    %d\t%.2f" %(closest_id, sim))

По мера Жаккара:
Q: compressibility transformation:
    ID	Коэффициент
    20	0.13
    0	0.00
Q: yawed cylinder:
    ID	Коэффициент
    60	0.14
    115	0.07

Косинустное расстояние:
Q: compressibility transformation:
    ID	Коэффициент
    20	0.54
    0	1.00
Q: yawed cylinder:
    ID	Коэффициент
    60	0.56
    61	0.75
